In [1]:
import numpy as np
import matplotlib.pyplot as plt
import timeit

In [2]:
import time

# @Decorator timeit
# Retorna o tempo de excução da função
# Não modifica a função.
# Para mais informações sobre decorators e timeit:
# https://medium.com/pythonhive/python-decorator-to-measure-the-execution-time-of-methods-fa04cb6bb36d
def timeit(method):
    def timed(*args, **kw):
        ts = time.time();
        result = method(*args, **kw);

        te = time.time();

        # Armazenar resultado
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__);
            kw['log_time'][name].append((te - ts) * 1000);
        else:
            print("%r  %2.5f ms" % (method.__name__, (te - ts) * 1000));
        return result
    return timed

In [ ]:
def f_circ(a, b, r):
  # Dado um circulo (x - a)^2 + (y - b)^2 = r^2
  # Temos três pontos x = (x1, x2, x3) e y = (y1, y2, y3)
  # encontre o quão a, b e r está distante.
  n = 3
  f = np.zeros([n], dtype = 'float64')
  
  R = r*r;
  f[0] = ((8.21 - a)*(8.21 - a)) + ((0.0 - b)*(0.0 - b)) - R
  f[1] = ((0.34 - a)*(0.34 - a)) + ((6.62 - b)*(6.62 - b)) - R
  f[2] = ((5.96 - a)*(5.96 - a)) + ((-1.12 - b)*(-1.12 - b)) - R
  return f

## A resolução da Equação da Circunferência

$(x - a)^2 + (y - b)^2 = R^2$

Temos três pontos x e três y $(x1, y1)...(x3,y3)$ que retornam o raio.

Encontra $a,b$ e $R$

In [ ]:
def newtonRapshon_ji(f, x, tol = 1.0e-9):
  # F(X) = 0
  def jacobiano(f, x):
    #     | f1/dx  f1/dy  f1/dz |
    # J = | f2/dx  f2/dy  f2/dz |
    #     | f3/dx  f3/dy  f3/dz |
    h = 1.0e-4
    n = x.shape[0]
    jac = np.zeros([n, n], dtype='float64')
    f0 = f(x) # f0 = { f1(x), f2(x), ..., fn(x) }
    for i in np.arange(n):
      aux = x[i]
      x[i] = aux + h  # x = {x0, ..., xi + h, ..., xn } ; avança x na direção ith
      f1 = f(x)       # f1 = { f1/di, f2/di, ..., fn/di }
      x[i] = aux
      jac[:, i] = (f1 - f0) / h
    
    return jac, f0
  
  for i in np.arange(30):
    jac, f0 = jacobiano(f, x)
  
    # the solutions is near enough of f(X) ~ 0
    if(np.sqrt(np.dot(f0, f0)) < tol):
      return x
    else:
      jac_inv = inverter_matrix(jac)
      
      # x_k_new = x_k - (jacobiana_inversa @ F(x_k)) 
      dx = jac_inv.dot(f0)
      x = x - dx
      if(1.0 > np.max(np.abs(x))):
        y = 1.0
      else:
        y = np.max(np.abs(x))
      
      # Se dx for indicar pouco deslocamento:
      if(np.sqrt(np.dot(dx, dx)) < tol*y):
        return x
  print('The solution is far yet.')
  return -1

In [ ]:
def f(x):
  # Intersecção entre circulo e hipérbole:
  # f1(x, y) = x^2 + y^2 - 3 = 0
  # f2(x, y) = xy - 1 = 0
  n = x.shape[0]
  f = np.zeros([n], dtype = 'float64')
  f[0] = (x[0]*x[0]) + (x[1]*x[1]) - 3
  f[1] = (x[0]*x[1]) - 1
  return f

In [ ]:
def test_jac_inv():
  x = np.array([0.5, 1.5], dtype='float64')
  bad_guess = np.array([1.5, 2.5], dtype='float64')


  startTimer()
  print(newtonRapshon_j(f, np.copy(x)))
  getTimer()

  startTimer()
  print(fsolve(f, x))
  getTimer()

  startTimer()
  print(newtonRapshon_j(f, np.copy(bad_guess)))
  getTimer()

  startTimer()
  print(fsolve(f, bad_guess))
  getTimer()

In [ ]:
def newtonRapshon_j(f, x, tol = 1.0e-9):
  # F(X) = 0
  # O critério de parada é a solução está próxima a zero,
  # Ou -1 se estiver muito longe.
  def jacobiano(f, x):
    #     | f1/dx  f1/dy  f1/dz |
    # J = | f2/dx  f2/dy  f2/dz |
    #     | f3/dx  f3/dy  f3/dz |
    h = 1.0e-4
    n = x.shape[0]
    jac = np.zeros([n, n], dtype='float64')
    f0 = f(x) # f0 = { f1(x), f2(x), ..., fn(x) }
    for i in np.arange(n):
      aux = x[i]
      x[i] = aux + h # x = {x0, ..., xi + h, ..., xn } ; avança x na direção ith
      f1 = f(x)       # f1 = { f1/di, f2/di, ..., fn/di }
      x[i] = aux
      jac[:, i] = (f1 - f0) / h
    
    return jac, f0
  
  for i in np.arange(30):
    jac, f0 = jacobiano(f, x)
    if(np.sqrt(np.dot(f0, f0)) < tol):
      return x
    else:
      A, b = gaussPivot(jac, -f0)
      dx = backSubstituion(A, b)
      x = x + dx
      if(1.0 > np.max(np.abs(x))):
        y = 1.0
      else:
        y = np.max(np.abs(x))
      
      if(np.sqrt(np.dot(dx, dx)) < tol*y):
        return x
  print('The solution is far yet.')
  return -1